In [17]:
from pysanger import *
import matplotlib.pyplot as plt

import os
from Bio import SeqIO

from tqdm.notebook import tqdm

with open("data/bfp-expression-control-mcherry-flag-hlpla-pcdna3-w37v.fasta") as file:
    for record in SeqIO.parse(file, "fasta"):
        template = record.seq

def tokenize_fp_make_id(inp):
    tokens = inp.replace('-', '_').split('_')
    user = tokens[0]
    construct = tokens[2]
    num = tokens[3]

    forward = 'Mcherry' in tokens
    reverse = 'WPRE' in tokens

    if not forward and not reverse:
        raise Exception('Neither forward nor reverse! Invalid filename.')
    else:
        if forward:
            primer = 'MCHERRY'
            direction = 'FORWARD'
            strand = 1
        else:
            primer = 'WPRE'
            direction = 'REVERSE'
            strand = -1

    id = f"{user}_{construct}_{num}_{primer}_{direction}"

    return id, strand

region = "read"
fp = 'PC_812288-1030_30_PC-442--Mcherry-Seq-Forward_C06.ab1'
id, strand = tokenize_fp_make_id(fp)
abidata = abi_to_dict(f"peter_data/{fp}")

if strand == 1:
    template_limits = (4660, 5250)
else:
    template_limits = (5000, 5740)

translation_limits = (4702, None)

consensus_seq_set = generate_consensusseq(abidata)

if strand == 1: 
    subject = consensus_seq_set[0]

if strand == -1:
    subject = consensus_seq_set[1] 

if template_limits is not None:
    template_start = max(0, template_limits[0]-1) #adjust for 0-based indexing
    if template_limits[1] is None:
        template_end = len(template)
    else:
        template_end = min(len(template), template_limits[1])

    if template_start >= template_end:
        raise ValueError("Invalid template limits specified! Template start point should be less than template end point.")
else:
    template_start = 0
    template_end = len(template)

if translation_limits is not None:
    translation_start = max(0, translation_limits[0]-1)
    if translation_limits[1] is None:
        translation_end = len(template)
    else:
        translation_end = min(len(template), translation_limits[1])

template = template[template_start:template_end]

alignments = pairwise2.align.globalms(template, subject, 2, 0, -10, -1, penalize_end_gaps=False)
atemplate = alignments[0][0]
asubject = alignments[0][1]

ts = len(atemplate) - len(atemplate.lstrip('-'))
te = len(atemplate) - len(atemplate.rstrip('-'))

ss = len(asubject) - len(asubject.lstrip('-'))
se = len(asubject) - len(asubject.rstrip('-'))

#modify to proper indices
te = len(atemplate) - te 
se = len(asubject)  - se

if region == "template":
    start, end = ts, te
elif region == 'read':
    start, end = ss, se
else:
    raise ValueError("Invalid region specified!")

asubject  = asubject[start:end]
atemplate = atemplate[start:end]

if len(asubject) != len(atemplate):
    raise ValueError("Subject and template lengths do not match after truncation!")

length = len(asubject)

if region == 'read':
    t_start_base = max(template_start, template_start+ss-ts)
    s_start_base = 0
elif region == 'template':
    t_start_base = template_start
    s_start_base = max(0, ts-ss)

In [33]:
remove_start_num = len(atemplate) - len(atemplate.lstrip('-'))
at = atemplate.lstrip('-')
asub = asubject[remove_start_num:]

starting_codon = 1
starting_base = t_start_base + 1 #return to 1 index

def remove_index(inp_str, remove):
    char_removed = 0
    for idx in range(len(inp_str)):
        if inp_str[idx] != '-':
            char_removed += 1
        
        if char_removed == remove:
            return idx + 1


if translation_start > t_start_base:
    remove = translation_start - t_start_base #need to cut to remove this many bases from the start of the template
else:
    remove = (translation_start - t_start_base) % 3#need to cut to remove this many bases from the start of the template
    codon = starting_codon - (remove/3) #rounds up

starting_base += remove

at = at[remove_index(at, remove):]
asub = asub[remove_index(at, remove):]

temp_chars = len(at.rstrip('-')) - at.rstrip('-').count('-') #number of characters in template minus number of gaps
temp_chars = temp_chars // 3 * 3 #round down to nearest multiple of 3

at = at[:remove_index(at, temp_chars)]
asub = asub[:remove_index(at, temp_chars)]

#cut end of at, asub

#cluster atemplate into 3 letter chunks from template
at_codons = []
asub_codons = []
template_nt_pos = []
codon_num = []

current_codon = starting_codon
current_nt_pos = starting_base
curr_template_codon = ''
curr_subject_codon = ''
n_template_nt = 0
curr_codon_pos = []

for tc, sc in zip(at, asub):
    curr_template_codon += tc
    curr_subject_codon += sc
    curr_codon_pos.append(current_nt_pos)
    
    if tc != '-':
        n_template_nt += 1
        current_nt_pos += 1
    
    if n_template_nt == 3:
        at_codons.append(curr_template_codon)
        asub_codons.append(curr_subject_codon)
        template_nt_pos.append(curr_codon_pos)
        codon_num.append(current_codon)
        
        curr_template_codon = ''
        curr_subject_codon = ''
        curr_codon_pos = []
        n_template_nt = 0
        current_codon += 1    

if n_template_nt != 0:
    raise ValueError('Something has gone wrong :(')

insertions = []
deletions = []
nt_mismatches = []
silent_mutations = []
missense_mutations = []
frameshift_mutations = []

for at_codon, as_codon, codon_pos, curr_codon_num in zip(at_codons, asub_codons, template_nt_pos, codon_num):
    for at_codon_nt, as_codon_nt, nt_pos in zip(at_codon, as_codon, codon_pos):
        if at_codon_nt == '-':
            insertions.append(f"{nt_pos}(ins{as_codon_nt.upper()})")
        elif as_codon_nt == '-':
            deletions.append(f"{nt_pos}(del{at_codon_nt.upper()})")
        elif at_codon_nt != as_codon_nt:
            nt_mismatches.append(f"{nt_pos}({at_codon_nt.upper()}->{as_codon_nt.upper()})")

    at_codon_s = at_codon.strip('-').upper()
    as_codon_s = as_codon.strip('-').upper()

    if len(as_codon_s) != 3 or len(at_codon_s) != 3:
        frameshift_mutations.append(f"{curr_codon_num}")
    else:
        #both codons are 3 nt long
        at_aa = translate([at_codon_s])[0]
        as_aa = translate([as_codon_s])[0]

        if at_codon_s != as_codon_s:
            if at_aa == as_aa:
                silent_mutations.append(f"{curr_codon_num}")
            else:
                missense_mutations.append(f"{at_aa}{curr_codon_num}{as_aa}")

insertions = "; ".join(insertions)
deletions = "; ".join(deletions)
nt_mismatches = "; ".join(nt_mismatches)
silent_mutations = "; ".join(silent_mutations)
missense_mutations = "; ".join(missense_mutations)
frameshift_mutations = "; ".join(frameshift_mutations)

stats = pd.DataFrame({'id': id, 'alignment_score': alignments[0].score, 'nt_insertions' : [insertions], 'nt_deletions': [deletions], 'nt_mismatches': [nt_mismatches], 'silent_mutations': [silent_mutations], 'missense_mutations': [missense_mutations], 'gap_mutations': [frameshift_mutations]})

In [30]:
translate([asub_codons[7]])

['L']

In [28]:
translate(asub_codons[7])

['-', '-', '-']

In [29]:
translate(at_codons[7])

['-', '-', '-']

In [19]:
stats

,id,alignment_score,nt_insertions,nt_deletions,nt_mismatches,silent_mutations,missense_mutations,gap_mutations
0,PC_1030_30_MCHERRY_FORWARD,1072.0,,,4723(A->C); 4724(G->T); 4725(C->G); 4732(T->C)...,8; 11; 35; 37; 48; 57; 84; 147,,


In [5]:
stats

,id,alignment_score,nt_insertions,nt_deletions,nt_mismatches,silent_mutations,missense_mutations,gap_mutations
0,PC_2090_90_WPRE_REVERSE,1414.0,,,9840(T->C); 9842(C->G); 10071(A->G); 10143(G->...,47; 48; 124; 148; 149; 167; 168; 206; 207; 239...,,


In [6]:
t_start_base

4999

In [2]:
import os
os.listdir('peter_data')

['PC_812288-1043_43_PC-442--Mcherry-Seq-Forward_D07.ab1',
 'PC_812288-2090_90_WPRE-R_H06.ab1',
 'PC_812288-3052_28_PC-442--Mcherry-Seq-Forward_D07.ab1',
 'PC_812288-1036_36_PC-442--Mcherry-Seq-Forward_C12.ab1',
 'PC_812288-1030_30_PC-442--Mcherry-Seq-Forward_C06.ab1',
 'PC_812288-1042_42_PC-442--Mcherry-Seq-Forward_D06.ab1',
 'PC_812288-4054_30_WPRE-R_F07.ab1',
 'PC_812288-2008_8_WPRE-R_A08.ab1',
 'PC_812288-2079_79_WPRE-R_G07.ab1',
 'PC_812288-2088_88_WPRE-R_H04.ab1',
 'PC_812288-4058_59_WPRE-R_B08.ab1',
 'PC_812288-4052_28_WPRE-R_D07.ab1',
 'PC_812288-2055_55_WPRE-R_E07.ab1',
 'PC_812288-2086_86_WPRE-R_H02.ab1',
 'PC_812288-2091_91_WPRE-R_H07.ab1',
 'PC_812288-2016_16_WPRE-R_B04.ab1',
 'PC_812288-1044_44_PC-442--Mcherry-Seq-Forward_D08.ab1',
 'PC_812288-1007_7_PC-442--Mcherry-Seq-Forward_A07.ab1',
 'PC_812288-4022_14_WPRE-R_F03.ab1',
 'PC_812288-1041_41_PC-442--Mcherry-Seq-Forward_D05.ab1',
 'PC_812288-2014_14_WPRE-R_B02.ab1',
 'PC_812288-1055_55_PC-442--Mcherry-Seq-Forward_E07.ab1',

In [3]:
from pysanger import *
import matplotlib.pyplot as plt

import os
from Bio import SeqIO

from tqdm.notebook import tqdm

def tokenize_fp_make_id(inp):
    tokens = inp.replace('-', '_').split('_')
    user = tokens[0]
    construct = tokens[2]
    num = tokens[3]

    forward = 'Mcherry' in tokens
    reverse = 'WPRE' in tokens

    if not forward and not reverse:
        raise Exception('Neither forward nor reverse! Invalid filename.')
    else:
        if forward:
            primer = 'MCHERRY'
            direction = 'FORWARD'
            strand = 1
        else:
            primer = 'WPRE'
            direction = 'REVERSE'
            strand = -1

    id = f"{user}_{construct}_{num}_{primer}_{direction}"

    return id, strand

#read in template plasmid
with open("data/bfp-expression-control-mcherry-flag-hlpla-pcdna3-w37v.fasta") as file:
    for record in SeqIO.parse(file, "fasta"):
        template = record.seq

os.makedirs('output', exist_ok=True)

for fp in tqdm(os.listdir('peter_data')):
    #abidata = abi_to_dict("data/PC_806488-574_2_PC-442--Mcherry-Seq-Forward_B10.ab1")
    id, strand = tokenize_fp_make_id(fp)
    abidata = abi_to_dict(f"peter_data/{fp}")

    if strand == 1:
        template_limits = (4660, 5250)
    else:
        template_limits = (5000, 5740)
    fig = visualize(id, abidata, template=template, strand=strand, fig=None, region="read", translation_limits = (4702, None), template_limits = template_limits)
    fig.savefig(f'output/{id}.png', bbox_inches = "tight")
    plt.close()

  0%|          | 0/324 [00:00<?, ?it/s]

AttributeError: 'tuple' object has no attribute 'savefig'

Error in callback <function flush_figures at 0x7f1df35d9c60> (for post_execute):


KeyboardInterrupt: 